In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np
import spacy
import string
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import collections
from IPython.display import HTML, IFrame
from textblob import TextBlob
from wordcloud import WordCloud
from tqdm import tqdm_notebook
from torchtext import data
import torch

In [3]:
# tokenizer = lambda x: str(x).translate(str.maketrans('', '', string.punctuation)).strip().split()

# Step one defination of our fields. 
TEXT = data.Field(sequential=True, lower=True, tokenize='spacy', include_lengths = True)
LABEL = data.Field(sequential=False, use_vocab=False)

print("loading from csv ...")
tv_datafields = [("review", TEXT), ("label", LABEL)]

# Step two construction our dataset.
train, valid, test = data.TabularDataset.splits(path='/content/drive/MyDrive/SENTIMENT',
                                                train="train.csv", validation="valid.csv",
                                                test="test_dataset.csv", format="csv",
                                                skip_header=True, fields=tv_datafields)
print(train[0].__dict__.keys())

loading from csv ...
dict_keys(['review', 'label'])


In [5]:
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train, max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

print("build vocab success...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step four construct our iterator to our dataset. 

print("construct iterator success...")
train_iter , valid_iter , test_iter = data.BucketIterator.splits((train,valid,test), device=device, 
                                                 batch_sizes=(32,32,32), sort_key=lambda x: len(x.review),
                                                 sort_within_batch=True, repeat=False)

build vocab success...
construct iterator success...


In [58]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # self.rnn = nn.RNN(embedding_dim, 
        #                    hidden_dim, 
        #                    num_layers=n_layers, 
        #                    bidirectional=bidirectional, 
        #                    dropout=dropout)
        self.rnn = nn.GRU(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(),enforce_sorted=False)
        
        packed_output, hidden = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [59]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [60]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,233,321 trainable parameters


In [61]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [62]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.0366,  0.8674, -1.5107,  ...,  0.5435,  0.9190, -0.5427],
        [ 0.2731, -0.4722,  0.3721,  ..., -0.2184, -1.0922,  0.0049],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.3322, -0.4683, -0.4141,  ..., -0.5160, -0.5486, -0.1394],
        [ 0.9698, -1.1250, -1.2575,  ..., -0.1520, -0.0730, -0.3884],
        [-0.2291,  0.2815, -0.2983,  ..., -0.4389,  0.0047,  0.4784]])

In [63]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.3322, -0.4683, -0.4141,  ..., -0.5160, -0.5486, -0.1394],
        [ 0.9698, -1.1250, -1.2575,  ..., -0.1520, -0.0730, -0.3884],
        [-0.2291,  0.2815, -0.2983,  ..., -0.4389,  0.0047,  0.4784]])


In [64]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [65]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [66]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [67]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.review
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label.float())
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [68]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.review
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label.float())
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [69]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [70]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 47s
	Train Loss: 0.536 | Train Acc: 71.45%
	 Val. Loss: 0.327 |  Val. Acc: 85.40%
Epoch: 02 | Epoch Time: 0m 46s
	Train Loss: 0.299 | Train Acc: 87.62%
	 Val. Loss: 0.247 |  Val. Acc: 89.87%
Epoch: 03 | Epoch Time: 0m 47s
	Train Loss: 0.218 | Train Acc: 91.34%
	 Val. Loss: 0.224 |  Val. Acc: 91.19%
Epoch: 04 | Epoch Time: 0m 46s
	Train Loss: 0.168 | Train Acc: 93.74%
	 Val. Loss: 0.236 |  Val. Acc: 90.99%
Epoch: 05 | Epoch Time: 0m 47s
	Train Loss: 0.134 | Train Acc: 95.25%
	 Val. Loss: 0.238 |  Val. Acc: 91.25%


In [ ]:
RNN_ACCURACY = 59%
GRU_ACCURACY = 91%

In [73]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.238 | Test Acc: 90.39%


In [74]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [82]:
predict_sentiment(model, "This film is not bad")

0.09289972484111786